## Project the Graph into Memory
This has to be done in order for the algorithm to act upon it. You cannot use the algorithm against a graph unless it is projected into memory first.

```sql
MATCH (source:v3)-[r:CONTAINS]->(target:v3)
RETURN gds.graph.project(
	'comm1',
	source,
	target,
	{
		sourceNodeProperties: source { },
		targetNodeProperties: target { },
		relationshipProperties: r { .weight }
	},
	{ undirectedRelationshipTypes: ['*'] }
)
```

## Memory Estimation
This gives an estimate of how much memory is needed to load and build the communities. 
Keep in mind the memory limit of Neo4j that's set in your local container or in OpenShift.

```sql
CALL gds.leiden.write.estimate('comm1', {writeProperty: 'communityId', randomSeed: 19})
YIELD nodeCount, relationshipCount, requiredMemory
```

## Stats
This can give a count of the number of communities this call is expected to generate.

```sql
CALL gds.leiden.stats('comm1', { 
  includeIntermediateCommunities: true,
  concurrency: 1, 
  randomSeed: 19 
})
YIELD communityCount
```


## Stream Results
You can display the results of a Leiden algorithm without actually writing to the database. This can be useful to examine the output without committing to any write operations.

This example uses intermediate communities. The 

```sql
CALL gds.leiden.stream('comm1', { 
  randomSeed: 19,
  includeIntermediateCommunities: true,
  concurrency: 1
})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).text AS text, communityId, intermediateCommunityIds
ORDER BY communityId ASC
```

## Write Results
To actually write the information on intermediate communities back to the database, you must run the write command.

```sql
CALL gds.leiden.write('comm1', {
  writeProperty: 'intermediateCommunities',
  randomSeed: 19,
  includeIntermediateCommunities: true,
  concurrency: 1
})
YIELD communityCount, modularity, modularities
```

The modularity scores here in the context of the Leiden algorithm (or other community detection algorithms) measure the quality of the community structure detected in the graph. Modularity is a scalar value between -1 and 1 that quantifies how well the graph is divided into communities. Here's what the modularity scores represent:
- A higher modularity score indicates that the communities are well-defined, meaning there are more edges within communities and fewer edges between communities than expected by chance.
- Typical modularity values range from 0 to 1:
  - Close to 1: Strong community structure (dense connections within communities, sparse connections between them).
  - Close to 0: Weak or no community structure.
  - Negative: The community structure is worse than random (rare in practice).
- The modularity typically increases with each iteration as the algorithm optimizes the community assignments, but it may plateau or slightly fluctuate near the end.